In [1]:
import copy

import cvrplib
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as rnd

from alns import ALNS, State
from alns.accept import RecordToRecordTravel
from alns.select import RouletteWheel
from alns.stop import MaxRuntime

In [2]:
%matplotlib inline

In [3]:
SEED = 1234

In [4]:
# data, bks = cvrplib.read('data/ORTEC-n242-k12.vrp', 'data/ORTEC-n242-k12.sol')
data = cvrplib.read('data/C101.txt')

print(data.name)
print(data.dimension)
print(data.capacity)
print(data.demands)
# print(type(data.distances))
print(data.service_times)

C101
101
200
[0, 10, 30, 10, 10, 10, 20, 20, 20, 10, 10, 10, 20, 30, 10, 40, 40, 20, 20, 10, 10, 20, 20, 10, 10, 40, 10, 10, 20, 10, 10, 20, 30, 40, 20, 10, 10, 20, 30, 20, 10, 10, 20, 10, 10, 10, 30, 10, 10, 10, 10, 10, 10, 20, 40, 10, 30, 40, 30, 10, 20, 10, 20, 50, 10, 10, 10, 10, 10, 10, 30, 20, 10, 10, 50, 20, 10, 10, 20, 10, 10, 30, 20, 10, 20, 30, 10, 20, 30, 10, 10, 10, 20, 40, 10, 30, 10, 30, 20, 10, 20]
[0, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90]


In [5]:
import alns_state.state as st

In [6]:
initial_state = st.nearest_neighbor(data)
initial_state.routes

[[21, 22, 23, 26, 28, 30, 27, 25, 24, 29, 20, 43, 42],
 [67, 65, 63, 62, 74, 72, 61, 64, 68, 69, 66],
 [5, 3, 4, 6, 8, 9, 11, 10, 7, 1, 2, 75, 91, 89],
 [47, 49, 52, 50, 51, 48, 45, 44, 40, 41, 46, 59, 57, 55],
 [90, 87, 86, 84, 85, 88, 83, 82, 81, 78],
 [98, 96, 95, 94, 93, 92, 97, 100, 99],
 [13, 17, 18, 19, 15, 14, 16, 12],
 [34, 36, 39, 38, 37, 35, 31, 33, 32],
 [54, 53, 56, 58, 60, 80, 79, 77, 73],
 [76, 71, 70]]

In [7]:
from alns_operator import removal as rm
# import operator.removal as rm

In [8]:
# curr = rm.random_removal(initial_state, rnd.RandomState(SEED))
# print(curr.cost)

In [15]:
curr = rm.remove_specific_customer(initial_state, 29)
print(curr.cost)

1040


In [10]:
sorted_cost = rm.sort_desc_cost_request(initial_state)
print(sorted_cost)

[(29, 10), (73, 8), (46, 7), (70, 7), (60, 6), (78, 6), (88, 6), (89, 6), (20, 5), (42, 5), (2, 4), (16, 4), (30, 4), (31, 4), (53, 4), (92, 4), (12, 3), (19, 3), (39, 3), (66, 3), (71, 3), (75, 3), (84, 3), (100, 3), (8, 2), (9, 2), (10, 2), (13, 2), (14, 2), (18, 2), (23, 2), (24, 2), (33, 2), (35, 2), (38, 2), (41, 2), (51, 2), (52, 2), (55, 2), (62, 2), (63, 2), (68, 2), (69, 2), (72, 2), (74, 2), (1, 1), (11, 1), (15, 1), (22, 1), (25, 1), (37, 1), (40, 1), (47, 1), (59, 1), (80, 1), (81, 1), (82, 1), (83, 1), (85, 1), (90, 1), (91, 1), (93, 1), (94, 1), (95, 1), (96, 1), (99, 1), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (17, 0), (21, 0), (26, 0), (27, 0), (28, 0), (32, 0), (34, 0), (36, 0), (43, 0), (44, 0), (45, 0), (48, 0), (49, 0), (50, 0), (54, 0), (56, 0), (57, 0), (58, 0), (61, 0), (64, 0), (65, 0), (67, 0), (76, 0), (77, 0), (79, 0), (86, 0), (87, 0), (97, 0), (98, 0)]


In [14]:
curr = rm.worst_removal(initial_state, rnd.RandomState(SEED))
print(curr.cost)
print(curr.routes)

1050
[[21, 22, 23, 26, 28, 30, 27, 25, 24, 29, 20, 43, 42], [67, 65, 63, 62, 74, 72, 61, 64, 68, 69, 66], [5, 3, 4, 6, 8, 9, 11, 10, 7, 1, 2, 75, 91, 89], [47, 49, 52, 50, 51, 48, 45, 44, 40, 41, 46, 59, 57, 55], [90, 87, 86, 84, 85, 88, 83, 82, 81, 78], [98, 96, 95, 94, 93, 92, 97, 100, 99], [13, 17, 18, 19, 15, 14, 16, 12], [34, 36, 39, 38, 37, 35, 31, 33, 32], [54, 53, 56, 58, 60, 80, 79, 77, 73], [76, 71, 70]]
